# Introduccion a brightway - pt. 2

En esta seccion hablaremos de los conceptos fundamentales de brigthway. Es importante aclarar que toda esta informacion esta disponible en linea en la pagina de documentacion:

https://docs.brightway.dev/en/latest/index.html

Este notebook se ejecutara en una nueva sesion, por lo que necesitamos instalar las dependencias una vez mas.

In [ ]:
!pip install bw2calc>=2.1 -q # Paquete de brightway
!pip install bw2data>=4.5 -q # Paquete de brightway
!pip install bw2io>=0.9.11 -q # Paquete de brightway
!pip install polars -q
!pip install pypardiso -q
!pip install seaborn>=0.13.2 -q

<div class="alert alert-block alert-warning">
⚠️ Debes restaurar la sesion!
</div>

Debemos descarg un archivo de respaldo que contiene ecoinvent. Para ellos tenemos que autenticar nuestro usuario de **gmail** que fue creado anteriormente.

In [ ]:
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
drive.CreateFile({'id': '1E3wPUOuRp13ucuNmq3557EuT3mszlmHB'}).GetContentFile('backup.tar.gz')

Verificamos

In [ ]:
!du -hs backup.tar.gz

In [ ]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
from rich import print

### Importar el backup del proyecto
Este modalidad no require mucha explicacion: El proyecto se carga nuevamente.

In [ ]:
bi.restore_project_directory(
    'backup.tar.gz',  # nombre del archivo, creado celdas arriba
    project_name='proyecto_ei', # Se puede elegir un nombre nuevo para el proyecto
    overwrite_existing = False
    )

## Importar bases de datos comerciales
Hemos aprendido a construir un modelo de ACV desde cero y de forma manual. Aunque esto resulta bastante util, en la realidad solemos combinar nuestros datos con aquellos provenientes de bases de datos comerciales. En esta seccion nos enfocaremos en la base de datos ecoinvent (v3.9), que es una de las mas utilizadas en el sector.

En la actualidad hay dos maneras de importar los datos de ecoinvent en nuestro proyecto:
- Leyendo los archivos ecospold2 crudos directamente del disco y convirtiendolos en una base de datos de brigthway.
- Utilizando la herramienta `import_ecoinvent_release` que descarga la base de datos desde un servidor remoto.

### Importando ecoinvent (crudo) desde el disco

Para este caso, es necesario haber descargado ecoinvent. Ecoinvent es distribuido en formato comprimido 7z, y contiene todas las actividades en formato ecospold2 (algo similar a XML). `bw2io` tiene funciones disenadas para interpretar la informacion, verificar que los `exchanges` sean correctos, y que los nodos de la biosfera existan en la base de datos 'biosphere3'.

In [ ]:
# Para importar, hay que seguir los siguentes pasos:
# 1. Leer los archivos XML e dejar que brigthway los interprete.
# db = bi.SingleOutputEcospold2Importer(dirpath='<datasets-folder>',db_name='ecoinvent39')

In [ ]:
# 2. Aplicar una serie de estrategias para asegurarse que no existe informacion corrupta y que la importacion es posible
# db.apply_strategies()

In [ ]:
# 3. Ecoinvent esta listo en la memoria pero aun no ha sido grabado en el disco.
# Hay que grabarlo en el disco.
# db.write_database()

Para verificar que ha sido importado correctamente, podemos repetir el ejercicio realizado con la base de datos 'biosphere3' de la anterior seccion.

In [ ]:
# bd.databases # Lista de las bases de datos

In [ ]:
# ei = bd.Database('ecoinvent39')
# len(ei) # Muestra la cantidad de elementos

### Importando ecoinvent desde un servidor remoto
Para este caso utilizamos la funcion `bw2io.import_ecoinvent_release` que se encarga de 1) instalar una biosfera, 2) instalar los metodos de impacto mas actuales, y 3) instalar la base de datos ecoinvent.
Como podran imaginar, requiere la autenticacion del usuario que debe poseer un cuenta de acceso ecoinvent

In [ ]:
# bw2io.import_ecoinvent_release(
#     version="3.9"
#     system_model="cutoff", # Otras opciones son: "consequential", "apos" y "EN15804"
#     username="xxxx", # Tu usuario
#     password="xxxx", # Tu clave
#     biosphere_name="biosphere" # Optional, puedes guardar la base de datos de la biosfera con otro nombre.
# )

### Explorando Ecoinvent

In [ ]:
import bw2data as bd
bd.projects.set_current("proyecto_ei")
ei = bd.Database('ecoinvent-3.9.1-cutoff')
seleccionado = ei.random() # Explora las actividades
# Primero exploramos los keys de la actividad seleccionada
print(list(seleccionado.keys()))

In [ ]:
# Podemos ver el contenido de todo el dataset.
print(seleccionado.as_dict())

Como pueden notar, el contenido de la actividad ecoinvent es bastante rica. Existen campos fuera de `name`, `code`,`location` y `unit` que son nuevos para nosotros, lo que demuestra que brightway es lo suficientemente flexible al definir una actividad.

Lo que vimos en la celda anterior describe a una actividad, pero aun no describe sus conexiones (`exchanges`). Para acceder a ellas, hay que utilizar las funciones `exchanges`, `technosphere` o `biosphere`, segun lo que se desee observar.

In [ ]:
# `exchanges` retorna un objeto the brightway que no es nativo de python
type(seleccionado.exchanges())

In [ ]:
# Si deseamos leerlo al estilo de una lista, hay que convertirlo en una lista.
for exchange in seleccionado.exchanges():
    print(exchange)
# print(list(seleccionado.exchanges()))

In [ ]:
# Si deseamos solo la tecnosfera, usamos la funcion correspondiente
for exchange in seleccionado.technosphere():
    print(exchange)
# print(list(seleccionado.technosphere()))

La impresion realizada en la celda de arriba nos muestra la informacion necesaria para poder construir las matrices. Sin embargo, brightway nos permite manipular el `exchange` y acceder a su metadata.

In [ ]:
# Seleccionamos el segundo `exchange`de la lista
exchange = list(seleccionado.technosphere())[1]
print(exchange.as_dict())

## Opciones de busqueda
Como podran imaginar, manipular una base de datos con tantas actividades (~21k) es bastante complicado. Podemos utilizar funciones nativas de python para realizar una busqueda.

In [ ]:
for x in ei:
  if x['name'] == 'transport, freight, lorry >32 metric ton, EURO5':
    print(x)

In [ ]:
# truck = [x for x in ei if x['name'] == 'transport, freight, lorry >32 metric ton, EURO5'][0]
# truck

Esta manera de buscar es mas 'pythonic'. Sin embargo, tambien puedes usar el buscador de brightway a traves de la funcion `search`.

In [ ]:
ei.search('transport, freight RoW >32 EURO5')

In [ ]:
ei.search?? # La funcion search prioriza algunos campos para hacer el filtro.

# Introduccion a brightway - pt. 3

En esta seccion hablaremos de los conceptos fundamentales de brigthway. Es importante aclarar que toda esta informacion esta disponible en linea en la pagina de documentacion:

https://docs.brightway.dev/en/latest/index.html

## Explorando las matrices
Ahora que sabemos como crear un actividad y metodos desde cero. Podemos concentrarnos en manipular las actividades que estan presentes en ecoinvent.
Para esta parte usaremos un proyecto que hemos preparado para ustedes que contiene una biosfera y tecnosfera compatible con ecoinvent v3.9

In [ ]:
# Re-import for this section if running independently
import bw2data as bd
import bw2io as bi
import bw2calc as bc
from rich import print

Tenemos dos bases de datos

In [ ]:
bd.databases

In [ ]:
# seleccionamos la base de datos ecoinvent y una actividad que tomaremos de ejemplo
ei = bd.Database("ecoinvent-3.9.1-cutoff")
harina = ei.search('fishmeal PE 65-67')[0]
harina

In [ ]:
# Elegimos un metodo que ya esta instalado y hace un LCA pero nos detenemos en la etapa de LCI
method=('IPCC 2021', 'climate change', 'global warming potential (GWP100)')
lca = bc.LCA({harina:1},method=method) # Instancia la clase
lca.lci() # calcula el inventario de ciclo de vida

Recordemos que ecoinvent tiene 21238 actividades
Entonces que dimensiones deberia tener la matriz de la tecnosfera?

In [ ]:
lca.technosphere_matrix.toarray().shape

Que dimensiones deberia tener el vector s?

In [ ]:
lca.supply_array

Si quisiera saber cuanto de 'anchoveta pescada en embarcaciones de madera'
se requiere en TOTAL para producir 1 kg de harina de pescado...

In [ ]:
anchoveta = ei.search('anchovy PE wooden')[1]
anchoveta

In [ ]:
# el lca.activity_dict me permite ubicar una actividad en la matriz.
lca.supply_array[lca.activity_dict[anchoveta.id]]

Ahora continuamos con el LCIA

In [ ]:
lca.lcia() # Calcula los impactos
print("El impacto es: ", lca.score)

## Analisis de contribuciones
Para entender las distintas contribuciones, tenemos que seguir utilizando el objeto LCA.
Este objeto mantiene los resultados del ACV en memoria

### Procesos mas importantes
Para listar los procesos que generan mas impactos utilizaremos el paquete `bw2analyzer` y `pandas`.

In [ ]:
import pandas as pd
import bw2analyzer as ba
# ba.ContributionAnalysis().annotated_top_processes(lca=lca) # dificil de visulizar
ba.ContributionAnalysis.annotated_top_processes??

In [ ]:
pd.DataFrame(
    [(x, y, z["name"]) for x, y, z in ba.ContributionAnalysis().annotated_top_processes(lca=lca)],
    columns=["score", "quantity", "name"]
)

### Emisiones mas importantes
De manera similar, podemos obtener el ranking de flujos ambiental que generan mayores impactos

In [ ]:
import pandas as pd
import bw2analyzer as ba
pd.DataFrame(
    [(x, y, z["name"]) for x, y, z in ba.ContributionAnalysis().annotated_top_emissions(lca=lca)],
    columns=["score", "quantity", "name"]
)

La importancia de las emisiones en el impacto tiene que ver con la cantidad y con los factores de caracterizacion.
Podemos listar estos factores para revisarlos

In [ ]:
for key, cf in bd.Method(method).load():
    # print(key, cf)
    print(bd.Database('biosphere3').get(key[1]))